In [203]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

### get the SA2 name and coe for the property

### read CSV
property_file_path = '../data/curated/property/property_4_lat_lon.csv'
property_data = pd.read_csv(property_file_path)

property_data['geometry'] = property_data.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
property_gdf = gpd.GeoDataFrame(property_data, geometry='geometry', crs='EPSG:4326')

### SA2 layer
file_path = '../data/extracted_files/G01_VIC_GDA2020.gpkg'
gdf_sa2 = gpd.read_file(file_path, layer='G01_SA2_2021_VIC')
gdf_sa2 = gdf_sa2.to_crs(epsg=4326)
merged_gdf = gpd.sjoin(property_gdf, gdf_sa2[['SA2_CODE_2021', 'SA2_NAME_2021', 'geometry']], how='left', op='within')
### save
output_file_path = '../data/curated/property/property4_sa2_data.csv'

merged_gdf.to_csv(output_file_path, index=False)
#print('saved')

/Users/qi/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3400: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


### read api and merge

In [68]:
p4 = pd.read_csv('../data/curated/property/property4_sa2_data.csv')
cbd_4 = pd.read_csv('../data/curated/external/API/4/closest_cbd_distance_final_4.csv')
gym_4 = pd.read_csv('../data/curated/external/API/4/closest_gym_distance_final_4.csv')
library_4 = pd.read_csv('../data/curated/external/API/4/closest_library_distance_final_4.csv')
police_4 = pd.read_csv('../data/curated/external/API/4/closest_police_distance_final_4.csv')
school_4 = pd.read_csv('../data/curated/external/API/4/closest_school_distance_final_4.csv')
station_4 = pd.read_csv('../data/curated/external/API/4/closest_station_distance_final_4.csv')
supermarket_4 = pd.read_csv('../data/curated/external/API/4/closest_supermarket_distance_final_4.csv')

In [69]:
pro_api_1 = pd.merge(p4, cbd_4, how='left',left_on='address', right_on='rent_address')
pro_api_1 = pro_api_1.drop(columns=['rent_address','closest_cbd'])

In [70]:
pro_api_2 = pd.merge(pro_api_1, gym_4, how='left',left_on='address', right_on='rent_address')
pro_api_2 = pro_api_2.drop(columns=['rent_address','closest_gym'])


In [71]:
pro_api_3 = pd.merge(pro_api_2, library_4, how='left',left_on='address', right_on='rent_address')
pro_api_3 = pro_api_3.drop(columns=['rent_address','closest_library'])

In [72]:
pro_api_4 = pd.merge(pro_api_3, police_4, how='left',left_on='address', right_on='rent_address')
pro_api_4 = pro_api_4.drop(columns=['rent_address','closest_police'])

In [73]:
pro_api_5 = pd.merge(pro_api_4, school_4, how='left',left_on='address', right_on='rent_address')
pro_api_5 = pro_api_5.drop(columns=['rent_address','closest_school'])

In [74]:
pro_api_6 = pd.merge(pro_api_5, station_4, how='left',left_on='address', right_on='rent_address')
pro_api_6 = pro_api_6.drop(columns=['rent_address','closest_station'])

In [204]:
pro_api_7 = pd.merge(pro_api_6, supermarket_4, how='left',left_on='address', right_on='rent_address')
pro_api_7 = pro_api_7.drop(columns=['rent_address','closest_supermarket','agentName','latitude','longitude','geometry','index_right'])

### read external data

In [131]:
price_post = pd.read_csv('../data/curated/external/unique_postcode_price.csv')

In [132]:
### merge the dataset with history price
merge_df = pd.merge(pro_api_7,price_post,how='left',on='postcode')
merge_df = merge_df.dropna()

In [166]:
### read the sa2 file
sa2 = pd.read_csv('../data/curated/external/SA2/sa2final.csv')
### merger the sa2 file with the pervious dataset
merge_df_1 = pd.merge(merge_df,sa2,how='left',left_on='SA2_CODE_2021',right_on='SA2 code')

In [177]:
### merge with the school postcode
uni_school = pd.read_csv('../data/raw/external/school/postcode_school.csv')
merge_df_2 = pd.merge(merge_df_1,uni_school,how='left',left_on='postcode',right_on='Address_Postcode')
merge_df_2 = merge_df_2.drop(['Address_Postcode'],axis=1)

In [178]:
merge_df_2 

,Unnamed: 0_x,price,address,beds,baths,parking,postcode,SA2_CODE_2021,SA2_NAME_2021,minimum_distance_cbd,...,SA2 code,SA2 name,2022 popluation,2023 popluation,ERP change %,Net overseas migration,Population density 2023 (persons/km2),Median_tot_prsnl_inc_weekly,2021 popluation,Number_of_Schools
0,6133,800.0,3508/120 A'beckett Street MELBOURNE VIC 3000,2,2,0,3000,214011375,Seaford (Vic.),49.98,...,214011375.0,Seaford (Vic.),17489.0,17764.0,1.6,184.0,1428.9,809.0,17215.0,4.0
1,6134,770.0,17 Michellan Court DONVALE VIC 3111,4,2,2,3111,211021261,Donvale - Park Orchards,24.33,...,211021261.0,Donvale - Park Orchards,16405.0,16511.0,0.6,242.0,793.7,826.0,16275.0,5.0
2,6136,800.0,1/121 Burke Road FERNTREE GULLY VIC 3156,4,2,2,3156,211011447,Ferntree Gully - North,40.48,...,211011447.0,Ferntree Gully - North,14234.0,14385.0,1.1,136.0,2059.4,775.0,14145.0,12.0
3,6137,795.0,1303/14 Queens Road MELBOURNE VIC 3004,1,1,1,3004,206051128,Albert Park,4.73,...,206051128.0,Albert Park,16177.0,16861.0,4.2,727.0,3606.9,1352.0,15825.0,4.0
4,6138,780.0,102/469 Riversdale Road HAWTHORN EAST VIC 3123,2,2,1,3123,207011152,Hawthorn East,8.90,...,207011152.0,Hawthorn East,16115.0,16878.0,4.7,681.0,4413.7,1228.0,15684.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1733,8169,3800.0,2301/15 Caravel Lane DOCKLANDS VIC 3008,5,6,5,3008,206041118,Docklands,3.43,...,206041118.0,Docklands,16491.0,17723.0,7.5,1552.0,7251.6,1182.0,15493.0,1.0
1734,8172,3900.0,1 Tarver St PORT MELBOURNE VIC 3207,4,3,4,3207,206051511,Port Melbourne Industrial,6.14,...,206051511.0,Port Melbourne Industrial,2229.0,2493.0,11.8,67.0,326.2,1346.0,1907.0,3.0
1735,8174,5000.0,14A Ethel Street MALVERN VIC 3144,4,4,2,3144,208041194,Malvern - Glen Iris,9.39,...,208041194.0,Malvern - Glen Iris,20559.0,21348.0,3.8,760.0,3468.3,1264.0,20239.0,3.0
1736,8175,5750.0,5604/1 Queensbridge Square SOUTHBANK VIC 3006,2,2,2,3006,206041508,Southbank (West) - South Wharf,2.33,...,206041508.0,Southbank (West) - South Wharf,6827.0,7418.0,8.7,681.0,8731.2,1227.0,6463.0,2.0


In [199]:
### get the crime form and divided into years
crime = pd.read_csv('../data/curated/crime_by_year.csv')
crime_2021 = crime[crime['Year'] == 2021]
crime_2021 = crime_2021.rename(columns={
    'Offence Count':'2021crime'
})
crime_2022 = crime[crime['Year'] == 2022]
crime_2022 = crime_2022.rename(columns={
    'Offence Count':'2022crime'
})
crime_2023 = crime[crime['Year'] == 2023]
crime_2023 = crime_2023.rename(columns={
    'Offence Count':'2023crime'
})

### merge them
merge_df_3 = pd.merge(merge_df_2,crime_2021,how='left',left_on='postcode',right_on='Postcode')
merge_df_3 = merge_df_3.drop(['Year','Postcode'],axis=1)
merge_df_4 = pd.merge(merge_df_3,crime_2022,how='left',left_on='postcode',right_on='Postcode')
merge_df_4 = merge_df_4.drop(['Year','Postcode'],axis=1)
merge_df_5 = pd.merge(merge_df_4,crime_2023,how='left',left_on='postcode',right_on='Postcode')
merge_df_5 = merge_df_5.drop(['Year','Postcode'],axis=1)

In [209]:
### drop useless columns
merge_df_all = merge_df_5.drop_duplicates('address')
merge_df_all = merge_df_all.drop(['Unnamed: 0_x','Unnamed: 0_y','SA2 code','SA2 name','Unnamed: 0'],axis=1)
merge_df_all = merge_df_all.dropna()

In [210]:
merge_df_all.to_csv('../data/curated/external/final_data_4.csv')